In [12]:
import os
from dotenv import load_dotenv
from langchain.llms import OpenAI

load_dotenv()
api_key = os.environ.get('OPENAI_API_KEY')
serpapi_key = os.environ.get('SERPAPI_API_KEY')


In [3]:
def simple_prompt(temperature: float):
    llm = OpenAI(openai_api_key=api_key, 
                 temperature=temperature)
    text = "What would be a good company name for a company that makes colorful socks?"
    print(llm(text))

simple_prompt(0.9)



Colorful Comfort Socks.


## Prompt Templates

In [8]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

def simple_prompt_template(
        product:str,
        temperature: float):
    llm = OpenAI(openai_api_key=api_key, 
                 temperature=temperature)
    prompt = PromptTemplate(
            input_variables=["product"],
            template="What is a good name for a company that makes {product}?",)
    print(f"prompt is:  {prompt.format(product=product)}")
    chain = LLMChain(llm=llm, prompt=prompt)
    return chain.run(product)


output = simple_prompt_template("AI consultancy", 0.2)
print("output is: ", output)

prompt is:  What is a good name for a company that makes AI consultancy?
output is:  

AI Advisors Inc.


## Agents

In [15]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI


def search_agent(temperature: float, run_agent: bool=False):
    llm = OpenAI(temperature=temperature)
    tools = load_tools(["serpapi", "llm-math"], llm=llm)
    agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
    if run_agent:
        agent.run("What was the high temperature in Munich yesterday in Centigrade? What is that number raised to the 2 power?")

search_agent(0.7)



> Entering new AgentExecutor chain...
 I need to find the temperature from yesterday and then do a calculation with it.
Action: Search
Action Input: "High temperature Munich yesterday Centigrade"
Observation: Munich Temperature Yesterday. Maximum temperature yesterday: 75 °F (at 2:50 pm) Minimum temperature yesterday: 46 °F (at 5:50 am) Average temperature ...
Thought: I need to convert the temperature from Fahrenheit to Centigrade.
Action: Calculator
Action Input: 75 Fahrenheit to Centigrade
Observation: Answer: 23.88888888888889
Thought: I now know the temperature in Centigrade and can calculate the power.
Action: Calculator
Action Input: 23.88888888888889^2
Observation: Answer: 570.679012345679
Thought: I now know the final answer.
Final Answer: The high temperature in Munich yesterday in Centigrade was 23.88888888888889 and that number raised to the 2 power is 570.679012345679.

> Finished chain.


## Memory

In [19]:
from langchain import OpenAI, ConversationChain

def conversation_bot(temperature: float,
                    input: str,
                    conv_bot: ConversationChain=None):
    llm = OpenAI(temperature=temperature)
    conversation = ConversationChain(llm=llm, verbose=True)
    if conv_bot is None:
        output = conversation.predict(input=input)
    else:
        output = conv_bot.predict(input=input)
    print(output)
    return conversation


conv_bot = conversation_bot(0.6, input())



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What would you like to eat
AI:

> Finished chain.
 I don't eat, but I can recommend some restaurants in your area. What type of food do you like?


In [22]:
conv_bot = conversation_bot(0.6, input(), conv_bot)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Why are you not familiar with it , i thought you were an information provider
AI:

> Finished chain.
 I am an information provider, but I'm not omniscient. I'm limited to the data and information that I have access to. I'm sorry I'm not familiar with it.


## Chat Models

In [24]:
from typing import Union, List
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

def chat_bot(temperature: float, 
             input:list, batch:bool=False):
    chat = ChatOpenAI(temperature=temperature)
    if not batch:
        output = chat(input)
    else:
        output = chat.generate(input)
    print(output)


chat_bot(0.75, [HumanMessage(content="Translate this sentence from English to French. I love programming.")])

content="J'adore la programmation." additional_kwargs={} example=False


In [25]:
chat_bot(0.75, [
    SystemMessage(content="You are a helpful assistant that translates English to French."),
    HumanMessage(content="I love programming.")
])

content="J'aime programmer." additional_kwargs={} example=False
